# Setup 

## Import packages 

In [1]:
# General 
import os 
import sys
import numpy as np
import pandas as pd 
# For running bash scripts from inside python ... 
import subprocess
# For manipulating string objects 
import re
# for generating any necessary directories
import pathlib 
# For manipulating list objects 
import itertools 

In [2]:
# For working with sequence objects 
from Bio.Seq import Seq

In [3]:
# For fetching sequences from Entrez 
from Bio import Entrez
from Bio import SeqIO

In [4]:
# For extracting features 
from Bio.SeqFeature import SeqFeature, FeatureLocation
# For creating SeqRecord objects 
from Bio.SeqRecord import SeqRecord

## Misc

In [5]:
Entrez.email = "kehaliwoldemichael@gmail.com"  # Always tell NCBI who you are

# Functions 

## Sequence

In [6]:
def seq_returnEntrez(sequenceID, retType):
    with Entrez.efetch(
        db="nucleotide", rettype=retType, retmode="text", id=sequenceID
    ) as handle:
        seqRecord = SeqIO.read(handle, "gb")  # using "gb" as an alias for "genbank"
        
    handle = Entrez.efetch(db="nucleotide", id=sequenceID, rettype=retType, retmode="text")
    
    return seqRecord, handle 

In [7]:
# Checks if continuous open reading frame by translating to stop ... 
def check_cORF(sequence):
    return len(sequence.translate(to_stop=True)) == len(sequence)/3

In [8]:
# Return sesRNAs that are in CDS 
def check_inCDS(sequence, searchSequence, isoForm, typeSes): 
    if typeSes == 'Reverse':
        return 0 != searchSequence[isoForm].seq.count(sequence.reverse_complement())
    elif typeSes == 'Complement':
        return 0 != searchSequence[isoForm].seq.count(sequence.complement())

In [9]:
# Checking for in frame TGG and ATG (both number and indices of occurances)
def return_inFrame(sequence, choice):
    # Definnig stop codons 
    stopCodons = ['TAG', 'TAA', 'TGA']
    
    # Generating list of codons in sequence 
    strSeq = str(sequence)
    codons = [strSeq for strSeq in re.split(r'(\w{3})', strSeq) if strSeq]
    
    # Number of in frame TGG and ATG 
    num_inF_TGG = codons.count('TGG')
    num_inF_ATG = codons.count('ATG')
    num_inF_Stop = codons.count(stopCodons[0]) + codons.count(stopCodons[1]) + codons.count(stopCodons[2])
    
    # Indices of TGG, ATG, and defined stop codons 
    indicesTGG = np.array([key for key, val in enumerate(codons) if val == 'TGG'])*3
    indicesATG = np.array([key for key, val in enumerate(codons) if val == 'ATG'])*3
    indiciesStop = np.array([key for key, val in enumerate(codons) if val in stopCodons])*3
    
    if choice == 'all': return num_inF_TGG, num_inF_ATG, num_inF_Stop, indicesTGG, indicesATG, indiciesStop 
    if choice == 'numTGG': return num_inF_TGG

In [10]:
# Return sesRNAs that are in CDS 
def return_inCDS(sesRNAs, CDS, isoForm, typeSes): 
    temp_cds_sesRNAs = []
    
    for sesRNA in sesRNAs: 
        if typeSes == 'Reverse':
            if 0 != CDS[isoForm].seq.count(sesRNA.reverse_complement()):
                temp_cds_sesRNAs.append(sesRNA)
        elif typeSes == 'Complement':
            if 0 != CDS[isoForm].seq.count(sesRNA.complement()):
                temp_cds_sesRNAs.append(sesRNA)
    
    return temp_cds_sesRNAs

## Metrics 

In [11]:
# Returns GC content 
def metric_gcContent(sequence):
    return (sequence.count("G") + sequence.count("C"))/(len(sequence))

# Sequence

## Loading sequences 

In [12]:
geneName = 'Tle4'
species = 'Rat'

martBasePath = os.getcwd() + '/Output/biomaRt/' + species 

In [13]:
# Loading sequences for reverse complement gene exons 
rC_fileName = martBasePath + '/Reverse_' + geneName + '.fasta'
rC_exon_records = list(SeqIO.parse(rC_fileName, "fasta"))
rC_exon_records  

[SeqRecord(seq=Seq('GACGTGGGGCTATGGAAGGGGATTAAACTCCACAACTGCAATGAGTATTGGAAA...GGA'), id='Tle4', name='Tle4', description='Tle4', dbxrefs=[]),
 SeqRecord(seq=Seq('CTCTTTGGTGATCATTGTCATGGTGCTTTTTCTCATCCTTAATTGGGAGGTGAG...CTG'), id='Tle4', name='Tle4', description='Tle4', dbxrefs=[]),
 SeqRecord(seq=Seq('CAGACAGTCCAGCTGAGAGACTGGGCTTTTGTTGCCAGGATGGCTGATGTCCCA...GCT'), id='Tle4', name='Tle4', description='Tle4', dbxrefs=[]),
 SeqRecord(seq=Seq('CTCACCAGAGTCTGGTTGTGCAGATCCCACACTGCGATGTTGCCATCACTGCAG...GTT'), id='Tle4', name='Tle4', description='Tle4', dbxrefs=[]),
 SeqRecord(seq=Seq('CTCTTTGGTGATCATTGTCATGGTGCTTTTTCTCATCCTTAATTGGGAGGTGAG...TTG'), id='Tle4', name='Tle4', description='Tle4', dbxrefs=[]),
 SeqRecord(seq=Seq('CGGGTGGCGCGTCTGCGGGTACATCTTGCTCAGGTCGCGAATCATCCACGCCGC...GCG'), id='Tle4', name='Tle4', description='Tle4', dbxrefs=[]),
 SeqRecord(seq=Seq('AGAAGTTATTTATGTTTAATGCTTAAAAGTCTGAATGCACAAACAATCTACCAT...GGA'), id='Tle4', name='Tle4', description='Tle4', dbxrefs=[]),
 SeqRecord(se

In [14]:
# Loading sequences for complement gene exons 
C_fileName = martBasePath + '/Complement_' + geneName + '.fasta'
C_exon_records = list(SeqIO.parse(C_fileName, "fasta"))
C_exon_records  

[SeqRecord(seq=Seq('AGGTTTCTTAGGAGTAGCCACGAATCGACACTGTAGAGACACCTACTGTTCATG...CAG'), id='Tle4', name='Tle4', description='Tle4', dbxrefs=[]),
 SeqRecord(seq=Seq('GTCGTTGTTGAGGTTCGAGTCGTAAATAGTGTACCTGTACCAGACGGACACGGA...CTC'), id='Tle4', name='Tle4', description='Tle4', dbxrefs=[]),
 SeqRecord(seq=Seq('TCGTATGAGGAAAGTGCAATCACGGCTACCAGTCTACGTCGGCCAGGGGAAAGG...GAC'), id='Tle4', name='Tle4', description='Tle4', dbxrefs=[]),
 SeqRecord(seq=Seq('TTGTCCCTATTGATGTAGGCGAGGACGTCTAACGAGGGACTACCGGCGTGGAAT...CTC'), id='Tle4', name='Tle4', description='Tle4', dbxrefs=[]),
 SeqRecord(seq=Seq('GTTGTTGAGGTTCGAGTCGTAAATAGTGTACCTGTACCAGACGGACACGGAGAC...CTC'), id='Tle4', name='Tle4', description='Tle4', dbxrefs=[]),
 SeqRecord(seq=Seq('GCGGCGGGAGTCACTGCGGGCGCCCCTTACGCCTCGGCCGGCGGGCGCGAGTCG...GGC'), id='Tle4', name='Tle4', description='Tle4', dbxrefs=[]),
 SeqRecord(seq=Seq('AGGTTTCTTAGGAGTAGCCACGAATCGACACTGTAGAGACACCTACTGTTCATG...AGA'), id='Tle4', name='Tle4', description='Tle4', dbxrefs=[]),
 SeqRecord(se

In [15]:
# Loading sequences for gene CDS 
CDS_fileName = martBasePath + '/CDS_' + geneName + '.fasta'
CDS = list(SeqIO.parse(CDS_fileName, "fasta"))
CDS  

[SeqRecord(seq=Seq('ATGTACCGCAGACGCCACCCGGCACCGCATCAGCCTGCTCAACCCTTCAAATTT...TAA'), id='Tle4', name='Tle4', description='Tle4', dbxrefs=[]),
 SeqRecord(seq=Seq('ATGATTCGCGACCTGAGCAAGATGTACCCGCAGACGCGCCACCCGGCACCGCAT...TAA'), id='Tle4', name='Tle4', description='Tle4', dbxrefs=[]),
 SeqRecord(seq=Seq('ATGATTCGCGACCTGAGCAAGATGTACCCGCAGACGCGCCACCCGGCACCGCAT...TGA'), id='Tle4', name='Tle4', description='Tle4', dbxrefs=[])]

In [16]:
len(CDS[0].seq)

2220

In [17]:
len(rC_exon_records[2].seq)

250

In [18]:
seq_record = rC_exon_records[1]

In [19]:
metric_gcContent(seq_record.seq)

0.5463414634146342

## Selecting sensor 

In [20]:
seq_record

SeqRecord(seq=Seq('CTCTTTGGTGATCATTGTCATGGTGCTTTTTCTCATCCTTAATTGGGAGGTGAG...CTG'), id='Tle4', name='Tle4', description='Tle4', dbxrefs=[])

In [21]:
seq = seq_record.seq
len(seq)

205

In [22]:
def generate_sesRNA(sequence, length, searchSequence, isoForm, typeSes):
    start = 0
    center = length/2
    
    global numPass, total, numTGG  
    numPass = 0 
    total = 0 
    numTGG = []
    
    sesSeq = []
    startSeq = []
    stopSeq = []
    gcContents = []
    firstTGG = []
    
    most_centralTGGs = []
    second_centralTGGs = []
    
    # For storing number of in frame TGG, ATG, and Stop codons 
    num_inF_TGGs = []
    num_inF_ATGs = []
    num_inF_Stops = []
    
    while(start <= (len(sequence) - length)):
        # Defining current sub-sequence to process 
        subsequence = sequence[start:(start+length)]
        
        # Genrating index of any stop codons 
        stopCodons = ['TAG', 'TAA', 'TGA']
        indiciesStop = []
        for codons in stopCodons:
            indiciesStop.extend([m.start() for m in re.finditer(codons, str(subsequence))])
        
        # GC content 
        gcContent = metric_gcContent(subsequence)*100
        # Index of last ATG and TGG 
        lastATG = 0 
        if(subsequence.count('ATG') != 0):
            lastATG = [m.start() for m in re.finditer('ATG', str(subsequence))][-1]
        if(subsequence.count('TGG') != 0):
            lastTGG = [m.start() for m in re.finditer('TGG', str(subsequence))][-1]
        # Getting indicies of TGG 
        indiciesTGG = [m.start() for m in re.finditer('TGG', str(subsequence))]
        # Generating arrays indicies for TGGs and stop codons 
        arrayStop = np.array(indiciesStop)
        arrayIndicies = np.array(indiciesTGG) 
        centralTGGs = arrayIndicies[abs(arrayIndicies - center) < 10]
        
        num_inF_TGG, num_inF_ATG, num_inF_Stop, indices_inF_TGG, indices_inF_ATG, indices_inF_Stop = return_inFrame(subsequence, 'all')
        numATG = subsequence.count('ATG')
        
        # Only proceed if passed 
        # cond1 = len(indiciesStop) < 4 
#         cond1 = check_cORF(subsequence)
        cond1 = num_inF_Stop <= 0
        
        cond2 = num_inF_TGG >= 2
        
        cond3 = num_inF_ATG == 0 
#         if num_inF_TGG != 0 and num_inF_ATG != 0:
#             cond3 = (min(indices_inF_TGG) > max(indices_inF_ATG)) # just making sure that all in frame ATG's upstream of all in frame TGG's 
#         else:
#             cond3 = num_inF_ATG == 0 
        
        cond4 = gcContent > 40
        cond5 = gcContent < 65
        
        # cond4 = lastATG < lastTGG 
        # Checking if TGG near center of subsequence 
        cond6 = any(abs(x - center) < 10 for x in indices_inF_TGG)
        
        # Checking if any central array is more than 10 by away from an in frame stop
#         cond7 = any((min(abs(arrayStop - i)) >= 20) for i in centralTGGs) # Original ... broken condition ... just as a check 
        if num_inF_Stop != 0:
            cond7 = any((min(abs(indices_inF_Stop - i)) >= 20) for i in centralTGGs)
        else:
            cond7 = True
        
        
        if(cond1 & cond2 & cond3 & cond4 & cond5 & cond6 & cond7):
            # Only include if in region of gene (currently in CDS) 
            if check_inCDS(subsequence, searchSequence, isoForm, typeSes):
                numPass += 1
                    
                numTGG.append(subsequence.count('TGG'))
                
                # Appending passesed subsequences 
                sesSeq.append(subsequence)
                # Appending indices of start for sesnsor (relative to start of exon)
                startSeq.append(start)
                stopSeq.append(start+length)
                gcContents.append(gcContent)
                
                firstTGG.append(indices_inF_TGG[0])
                
                sorted_TGGs = list((np.array(sorted(indices_inF_TGG - length/2, key = abs)) + (length/2)))
                most_centralTGGs.append(int(sorted_TGGs[0]))
                second_centralTGGs.append(int(sorted_TGGs[1]))
                
                # Appending number of in frame TGGs, ATGs, and Stop codons 
                num_inF_TGGs.append(num_inF_TGG)
                num_inF_ATGs.append(num_inF_ATG)
                num_inF_Stops.append(num_inF_Stop)
            
        total += 1 
        # Updating start index 
        start += 1 
    
    return sesSeq, startSeq, stopSeq, firstTGG, most_centralTGGs, second_centralTGGs, num_inF_TGGs, num_inF_ATGs, num_inF_Stops, gcContents 

In [23]:
sesRNAs = generate_sesRNA(seq, 204, CDS, 0, 'Reverse')
sesRNAs

([], [], [], [], [], [], [], [], [], [])

In [24]:
len(sesRNAs)

10

In [25]:
# Returns sesRNAs for each exon
def generate_sesRNAs_multiExon(exon_records, length, searchSequence, isoForm, typeSes):
    tempAll_sesRNAs = []
    tempAll_startSeq = []
    tempAll_stopSeq = []
    temp_num_inF_TGGs = []
    temp_all_firstTGG = []
    temp_all_centralTGG = []
    temp_all_second_central_TGG = []
    temp_num_inF_ATGs = []
    temp_num_inF_Stops = []
    temp_all_gcContents = []
    
    current = 1
    exon_number = []
    for record in exon_records:
        tempSeq = record.seq 
        temp_sesRNAs, temp_startSeq, temp_stopSeq, single_firstTGG, single_centralTGG, single_second_centralTGG, single_num_inF_TGGs, single_num_inF_ATGs, single_num_inF_Stops, temp_gcContents = generate_sesRNA(tempSeq, length, searchSequence, isoForm, typeSes)
        
        tempAll_sesRNAs.extend(temp_sesRNAs)
        
        tempAll_startSeq.extend(temp_startSeq)
        tempAll_stopSeq.extend(temp_stopSeq)
        
        temp_all_firstTGG.extend(single_firstTGG)
        temp_all_centralTGG.extend(single_centralTGG)
        temp_all_second_central_TGG.extend(single_second_centralTGG)
        
        
        temp_num_inF_TGGs.extend(single_num_inF_TGGs)
        temp_num_inF_ATGs.extend(single_num_inF_ATGs)
        temp_num_inF_Stops.extend(single_num_inF_Stops)
        
        exon_number.extend(np.full(len(temp_sesRNAs), current))
        temp_all_gcContents.extend(temp_gcContents)
        
        current += 1
        # Printing number of passed sequences for current exon 
        print(len(temp_sesRNAs))
        
    allMetrics = {'Exon':exon_number, 'StartSeq':tempAll_startSeq, 'StopSeq':tempAll_stopSeq, 
                  'firstTGG': temp_all_firstTGG, 'centralTGG': temp_all_centralTGG, 'second_cTGG': temp_all_second_central_TGG,
                  'numTGG':temp_num_inF_TGGs, 'numATG':temp_num_inF_ATGs, 
                  'numStop':temp_num_inF_Stops, 'gcContent':temp_all_gcContents}
    sequenceMetrics = pd.DataFrame(allMetrics)
    
    return tempAll_sesRNAs, sequenceMetrics 

In [30]:
CDS

[SeqRecord(seq=Seq('ATGTACCGCAGACGCCACCCGGCACCGCATCAGCCTGCTCAACCCTTCAAATTT...TAA'), id='Tle4', name='Tle4', description='Tle4', dbxrefs=[]),
 SeqRecord(seq=Seq('ATGATTCGCGACCTGAGCAAGATGTACCCGCAGACGCGCCACCCGGCACCGCAT...TAA'), id='Tle4', name='Tle4', description='Tle4', dbxrefs=[]),
 SeqRecord(seq=Seq('ATGATTCGCGACCTGAGCAAGATGTACCCGCAGACGCGCCACCCGGCACCGCAT...TGA'), id='Tle4', name='Tle4', description='Tle4', dbxrefs=[])]

In [34]:
rC_multiExon_sesRNAs, sequenceMetrics = generate_sesRNAs_multiExon(rC_exon_records, 228, CDS, 0, 'Reverse')

0
0
0
0
0
0
0
0
0


In [27]:
sequenceMetrics

,Exon,StartSeq,StopSeq,firstTGG,centralTGG,second_cTGG,numTGG,numATG,numStop,gcContent


In [28]:
all_indices_inF_TGG = []
all_indices_inF_ATG = []

for sequence in rC_multiExon_sesRNAs:
    num_inF_TGG, num_inF_ATG, num_inF_Stops, indicesTGG, indicesATG, indicesStops = return_inFrame(sequence, 'all')
    all_indices_inF_TGG.append(indicesTGG)
    all_indices_inF_ATG.append(indicesATG)
    print(indicesTGG)
    # print(num_inF_TGG)

In [29]:
max(sequenceMetrics['StartSeq'])

ValueError: max() arg is an empty sequence

In [ ]:
# Just looking at 'window' in which sesRNAs are being produced 
max(sequenceMetrics['StopSeq']) - min(sequenceMetrics['StartSeq'])

In [ ]:
# Checking GC content of sesRNAs 
for sequence in rC_multiExon_sesRNAs:
    print(metric_gcContent(sequence))

In [ ]:
C_multiExon_sesRNAs, C_multi_startSeq, C_num_inF_TGGs, C_num_inF_ATGs, C_num_inF_Stops = generate_sesRNAs_multiExon(C_exon_records, 204, CDS, 0, 'Complement')

In [ ]:
C_multiExon_sesRNAs

In [ ]:
# Just additional check if in CDS 
cds_sesRNAs = return_inCDS(rC_multiExon_sesRNAs, CDS, 0, 'Reverse')
cds_sesRNAs

In [ ]:
for i in range(200, 300):
    if(i%3 == 0):
        print(i)

In [ ]:
testSeq = sesRNAs[0]
testSeq

In [ ]:
str(testSeq)

In [ ]:
?split()

In [ ]:
subsequence = str(testSeq)
codons = [subsequence for subsequence in re.split(r'(\w{3})', subsequence) if subsequence]

In [ ]:
tempSeq = Seq('GTTCTCCTTCAGCACCTGCTCCAGCGGCGCATGCAAGCGCTCCTTATGGGGATAGGAAGCTGGGTGGGGGAACTTGTCCGCAGTCAGGCTGGCCAGTTTGGCATTCTCCAGCAGAAAAAGCTTGGGGTGAGCAGCCAGGGAAGTGGGGGCCTGTGCGTTGAGGAGGCCAGATGGGAAAAGGTGGCCTCCGAGGAGCTCCGATGG')

In [ ]:
check_cORF(tempSeq)

In [ ]:
coding_dna = Seq("ATGGCCATTGTAATGGGCCGCTGAAAGGGTGCCCGATAG")

In [ ]:
check_cORF(coding_dna)

# Outputing sesRNA

In [ ]:
# Generating BioPython directory if does not exist 
pathlib.Path('Output/BioPython').mkdir(parents=True, exist_ok=True)

# Generate SeqRecord object for each sequence and append to list 
outputID = geneName + '_sesRNA'
outputDescription = "sesRNA for " + geneName

# Generating sequence record objects (for seperate storage)
outputSeqMulti_DNA = []
outputSeqMulti_RNA = []
for i in rC_multiExon_sesRNAs:
    outputSeqMulti.append(SeqRecord(i, id = outputID, description = outputDescription))
    
# Write output fasta files 
version = "V6"
outputName = "Output/BioPython/" + geneName + "_sesRNA_" + version + ".fasta" 
with open(outputName, "w") as output_handle:
    SeqIO.write(outputSeqMulti, output_handle, "fasta")

In [ ]:
# Given sequence ... converts to in frame TGGs to TAGs and in frame stops so that first 'T' becomes 'G'
# Had to be careful to only work with in frame codons ... initally had made the mistake to just use string.replace ... this would change out of frame codons as well 
def convert_DNA(sequence, numberConvert):
    # Converting to string object for manipulation 
    strSeq = str(sequence)
    # Generating in frame object variables 
    num_inF_TGG, num_inF_ATG, num_inF_Stop, indicesTGG, indicesATG, indicesStop = return_inFrame(Seq(strSeq), 'all')
    print(num_inF_TGG)
    # print(num_inF_Stop)

    # Replacing in frame stop codons in sequence 
    for stop in indicesStop: 
        stopPairs = [("TAG", "GAG"), ("TAA", "GAA"), ("TGA", "GGA")]
        stopSeq = strSeq[stop:stop+3]
        [stopSeq := stopSeq.replace(a, b) for a, b in stopPairs]
        strSeq = strSeq[:stop] + stopSeq + strSeq[stop+3:]
    
    # Setting number convert to all if 'All' selected as number of TGG to convert 
    if numberConvert == 'All': numberConvert = num_inF_TGG
    # Converting TGG's ... up to number set ... and in order from starting with most central 
    # Sorts indicees by distance from center 
    sorted_indices_centralTGG = np.array(sorted(indicesTGG - (len(strSeq)/2), key = abs)) + (len(strSeq)/2)
    # Converts in frame TGG's ... starting from most central TGG ... up to limit set by numberConvert 
    for i in range(numberConvert):
        currentIndex = int(sorted_indices_centralTGG[i])
        strSeq = strSeq[:currentIndex] + 'TAG' + strSeq[currentIndex+3:]
    # Returns RNA 
    return Seq(strSeq).transcribe()

In [ ]:
testSeq = Seq('TGGGAGTAGTGGTGGTAATGA')
testStr = str(testSeq)

In [ ]:
convert_DNA(testSeq, 1)

In [ ]:
# Test that all and numbering is working 
convert_DNA(testSeq, 3) == convert_DNA(testSeq, 'All')

In [ ]:
convert_DNA(rC_multiExon_sesRNAs[0], 'All')

In [ ]:
# Testing that continious reading frame being produced if number set to 0 
len(convert_DNA(rC_multiExon_sesRNAs[0], 0).translate(to_stop = True)) == len(rC_multiExon_sesRNAs[0])/3

In [ ]:
# Checking that right number of TGG being converted 
return_inFrame(convert_DNA(rC_multiExon_sesRNAs[0], 'All').back_transcribe(), 'numTGG')

In [ ]:
convert_DNA(testSeq, 1) - (len(testSeq)/2)

In [ ]:
abs(convert_DNA(testSeq, 1) - (len(testSeq)/2))

In [ ]:
np.sort(abs(convert_DNA(testSeq, 1) - (len(testSeq)/2))) + (len(testSeq)/2)

In [ ]:
np.array(sorted(convert_DNA(testSeq, 1) - (len(testSeq)/2), key = abs)) + (len(testSeq)/2)

In [ ]:
# Function for saving both original template DNA sequence and the converted RNA given list of sesRNAs Seq objects
# Could change later to make for separate output files ... have not decided yet whether to just put them together into one labeled fasta file 
def save_sesRNAs(sequences_sesRNAs, geneName, version, numConvertTGG):
    # Generating BioPython directory if does not exist 
    pathlib.Path('Output/BioPython').mkdir(parents=True, exist_ok=True)
    
    # Defining save name and description for outputs (original DNA and converted RNA)
    DNA_outputID = geneName + '_sesRNA'
    DNA_outputDescription = "sesRNA DNA original for " + geneName
    convertedRNA_outputID = geneName + '_sesRNA'
    convertedRNA_outputDescription = "sesRNA converted RNA for " + geneName
    
    # Creating empty list for storing sequences 
    outputSeqMulti_DNA = []
    outputSeqMulti_convertedRNA= []
    
    # Generating SeqRecord objects in preparation for writing FASTA files ... edited description to include number of sequence
    i = 1
    for sequence in sequences_sesRNAs:
        outputSeqMulti_DNA.append(SeqRecord(sequence, id = DNA_outputID+str(i), description = DNA_outputDescription))
        outputSeqMulti_convertedRNA.append(SeqRecord(convert_DNA(sequence, numberConvertTGG), id = convertedRNA_outputID+str(i), description = convertedRNA_outputDescription))
        i += 1
        
    # Write output fasta files 
    DNA_outputName = "Output/BioPython/" + geneName + "_sesRNA_" + version + ".fasta" 
    convertedRNA_outputName = "Output/BioPython/" + geneName + "_sesRNA_convertedRNA_" + version + ".fasta" 
    
    with open(outputName, "w") as output_handle:
        # Writing original template DNA files 
        SeqIO.write(outputSeqMulti_DNA, output_handle, "fasta")
        # Writing orginal template RNA files 
        SeqIO.write(outputSeqMulti_convertedRNA, output_handle, "fasta")

# Secondary structure 

## Working 

In [ ]:
# Leading RNAfold as RNA 
sys.path.append("/home/user1/Dropbox/Research/Neurobiology_PhD/Rotations/Huang/Projects/CellReadR/Packages/ViennaRNA_Python3/usr/lib/python3.9/site-packages/RNA")
import _RNA as RNA

In [ ]:
# Generating Temp 
pathlib.Path('Output/BioPython/Temp').mkdir(parents=True, exist_ok=True)

In [ ]:
# Converting to RNA for calculating secondary structure 
sesRNAs_RNA = []
for i in range(len(rC_multiExon_sesRNAs)):
    sesRNAs_RNA.append(rC_multiExon_sesRNAs[i].transcribe())

In [ ]:
sesRNAs_RNA

In [ ]:
str(sesRNAs_RNA[0])

In [ ]:
# Just making sure to clear Temp folder before starting 
os.system('rm -rf Output/BioPython/Temp/*')

In [ ]:
# Writing sequences as seperate fasta files 
i = 1
all_mfe = []

for sesRNA in sesRNAs_RNA:
    (ss, mfe) = RNA.fold(str(sesRNAs_RNA[i-1]))
    all_mfe.append(mfe)
    
    # Making sure that single digit number stast with 0 so that files processed in order 
    if i < 10: numSes = '0' + str(i)
    else: numSes = str(i)
        
    # Defining output name 
    outputName = geneName + '_' + numSes
    outputDescription = "sesRNA #" + numSes
    i += 1
    
    outputRecord = SeqRecord(sesRNA, id = outputName, description = outputDescription)
    outputFull = 'Output/BioPython/Temp/' + outputName + '.fasta'
    
    with open(outputFull, "w") as output_handle:
        SeqIO.write(outputRecord, output_handle, "fasta")

In [ ]:
all_mfe

In [ ]:
# Add as column in sequence metrics dataframe 
sequenceMetrics['mfe'] = all_mfe
sequenceMetrics

In [ ]:
len(rC_multiExon_sesRNAs)

In [ ]:
# Call RNAfold on each sequence of output 
rnaFold_prob = []

intarna_columns = ['E', 'start', 'end1', 'start2', 'end2']
all_sorted_intarna = []
useful_intarna =  pd.Dataframe(columns = intarna_columns)

pathTemp = '/home/user1/Dropbox/Research/Neurobiology_PhD/Rotations/Huang/Projects/CellReadR/Code/Output/BioPython/Temp'
pathOutTempFold = pathTemp + '/temp.out'
pathOutTempIntaRNA = pathTemp + '/temp.csv'

# sorting files in output of scandir 
for entry in sorted(os.scandir(pathTemp), key=lambda e: e.name):
    # For checking which file currently working on (not in order for some reason) 
    print(entry.path)
    
    # Defining command for RNAfold 
    commandFold = 'RNAfold -p -d2 --noLP < ' + entry.path + ' > ' + pathOutTempFold    
    # Generating RNAfold predictions 
    generateProb = subprocess.run(commandFold, shell=True, stdout=subprocess.PIPE)
    
    # Defining command for IntaRNA
    numThreads = 10
    commandInta = 'IntaRNA -t ' + CDS_fileName + ' -q ' + entry.path + ' --threads ' + str(numThreads) + ' --outMode=C > ' + pathOutTempIntaRNA
    # Generating IntaRNA predictions 
    os.system(commandInta)

    # Moving to Temp directory to work on fasta files 
    currentWD = os.getcwd()
    os.chdir('Output/BioPython/Temp')
    
    # Running script for getting probabilities from RNAfold output file (added to ArchBin btw)
    readProb = subprocess.Popen("rnaFold_prob.sh", shell=True, stdout=subprocess.PIPE)
    returnedProb = readProb.stdout.read()
    # Waiting for last command to finish before storing value in temp.out file 
    readProb.wait()
    # Append frequences ... convert to percentage 
    rnaFold_prob.append(float(returnedProb)*100)
    
    # Reading in csv file with information 
    intarnaOutput = pd.read_csv(outputPath, sep = ';')
    # Saving pd output as list of lists ... would be a pain to compute multiple times if desired 
    sorted_intarna = intarnaOutput.sort_values('E', ascending=True)
    # Concatinating select columuns of dataframe for inclusion into sequence metrics 
    all_sorted_intarna.append(sorted_intarna)i
    pd.concat([useful_intarna, sorted_intarna[intarna_columns]])
    
    # Removing temp.out after finishing each run 
    os.system('rm -rf temp.out')
    os.system('rm -rf temp.csv')
    # Return to initial working directory 
    os.chdir(currentWD)
    # Removing files generated by RNAfold 
    os.system('rm -rf *ss.ps')
    os.system('rm -rf *dp.ps')

In [ ]:
outputPath = os.getcwd() + '/' + outputFile

In [ ]:
outputPath

In [ ]:
intarnaOutput = pd.read_csv(outputPath, sep = ';')

In [ ]:
intarnaOutput

In [ ]:
rnaFold_prob

In [ ]:
# Add as column in sequence metrics dataframe 
sequenceMetrics['rnaFoldProb'] = rnaFold_prob
# Concatenate IntaRNA output as additional columns 
pd.concat(sequenceMetrics, useful_intarna, axis = 1)
sequenceMetrics

In [ ]:
temp_metrics.sort_values('rnaFoldProb', ascending=True).iloc[0]

In [ ]:
sortedMetrics.loc[10, :]

In [ ]:
sortedMetrics

In [ ]:
sortedMetrics[['Exon', 'numTGG']]

## Not working or not being used 

In [ ]:
sys.path.append("/usr/share/ViennaRNA")

In [ ]:
# Leading RNAfold as RNA 
import sys
sys.path.append("/usr/lib/python3.9/site-packages/RNA")
import _RNA as RNA

In [ ]:
import pyseqlib

In [ ]:
from pyseqlib import pyRNAfold

In [ ]:
pyRNAfold i

In [ ]:
pyseqlib.fold_compound(sequence)

In [ ]:
RNA.fold_compound(sequence)

In [ ]:
sys.path.append("/home/user1/Dropbox/Research/Neurobiology_PhD/Rotations/Huang/Projects/CellReadR/Packages/RNA")
import _RNA as RNA

In [ ]:
sys.path.append("/home/user1/Dropbox/Research/Neurobiology_PhD/Rotations/Huang/Projects/CellReadR/Code/Functions")
import RNA_Fold

In [ ]:
md = RNA.md()

In [ ]:
import example

In [ ]:
sequence = "CGCAGGGAUACCCGCG"
 
# create new fold_compound object
fc = RNA.fold_compound(sequence)
 
# compute minimum free energy (mfe) and corresponding structure
(ss, mfe) = fc.mfe()
 

In [ ]:
# The RNA sequence
seq = "GAGUAGUGGAACCAGGCUAUGUUUGUGACUCGCAGACUAACA"
 
# compute minimum free energy (MFE) and corresponding structure
(ss, mfe) = RNA.fold(seq)

In [ ]:
import pickle 

In [ ]:
mfe

In [ ]:
pathOuput = "/home/user1/Dropbox/Research/Neurobiology_PhD/Rotations/Huang/Projects/CellReadR/Code/Functions/seqObject.p"
pickle.dump(testSeq, open(pathOuput, 'wb'))

In [ ]:
# Running script for getting probabilities from RNAfold output file (added to ArchBin btw)
pathFuncPython = '/home/user1/Dropbox/Research/Neurobiology_PhD/Rotations/Huang/Projects/CellReadR/Code/Functions/'
# command = 'python ' + '/home/user1/Dropbox/Research/Neurobiology_PhD/Rotations/Huang/Projects/CellReadR/Code/Functions/RNA_Fold.py'
# command = 'cd ' + pathFuncPython + ' | ./RNA_Fold.py'
# command = 'cd ' + pathFuncPython + ' | ls -a'

command = 'ls -a'
readProb = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE)
returnedProb = readProb.stdout.read()

In [ ]:
returnedProb

In [ ]:
command 

In [ ]:
str(rC_multiExon_sesRNAs[0])

In [ ]:
testSeq = str(rC_multiExon_sesRNAs[0])
testSeq

In [ ]:
RNA.fold(testSeq)

In [ ]:
def compute_mfeFreq(sequence):
    # create a fold_compound object for the current sequence
    fc = RNA.fold_compound(sequence)

    # compute the MFE and corresponding structure
    (mfe_struct, mfe) = fc.mfe()


    # compute partition function
    (bp_propensity, dG) = fc.pf()

    # compute frequency of MFE structure (the 'hard' way)
    kT = RNA.exp_param().kT / 1000.

    prob_mfe = math.exp((dG - mfe) / kT)
    
    return prob_mfe

In [ ]:
rC_multi_startSeq

In [ ]:
rC_multiExon_sesRNAs

In [ ]:
str(rC_multiExon_sesRNAs[8])

In [ ]:
str(rC_multiExon_sesRNAs[18])

In [ ]:
from seqfold import dg, dg_cache, fold

In [ ]:
# just returns minimum free energy
dg("GGGAGGTCGTTACATCTGGGTAACACCGGTACTGATCCGGTGACCTCCC", temp = 37.0)  # -12.94

# Test Intarna (test strength of binding, off-target, ...)  

In [ ]:
import intarnapvalue

In [ ]:
testSeq = str(rC_multiExon_sesRNAs[0])
testSeq

In [ ]:
target = str(CDS[0].seq)
target 

In [ ]:
from intarnapvalue.intarna_pvalue import IntaRNApvalue

In [ ]:
?IntaRNApvalue

In [ ]:
?intarna_pvalue

In [ ]:
IntaRNApvalue(['--query', testSeq, '--target', target])

In [ ]:
IntaRNApvalue['-q', 'AGGAUG', '-t', 'UUUAUCGUU', '-s', '10', '-m', 'b', '-d', 'gauss', '--threads', '3']

In [ ]:
command = 'python -m intarnapvalue --query GCUGAAAAACAUAACCCAUAAAAUGCUAGCUGUACCAGGAACCA --target GGUUUCUUCGCCUCUGCGUUCACCAAAGUGUUCACCC -s 10 --shuffle-mode b --threads 0' 

In [ ]:
readProb = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE)
readProb.wait()
returnedProb = readProb.stdout.read()
returnedProb

In [ ]:
# Running script for getting probabilities from RNAfold output file (added to ArchBin btw)
readProb = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE)
returnedProb = readProb.stdout.read()
# Waiting for last command to finish before storing value in temp.out file 
readProb.wait()

In [ ]:
pd.read_csv('/home/user1/Dropbox/Research/Neurobiology_PhD/Rotations/Huang/Projects/CellReadR/Code/Functions/output.csv', sep = ';')

In [ ]:
pd.read_csv('/home/user1/Dropbox/Research/Neurobiology_PhD/Rotations/Huang/Projects/CellReadR/Code/Functions/output.csv', sep = ';')

In [ ]:
pd.read_csv('/home/user1/Dropbox/Research/Neurobiology_PhD/Rotations/Huang/Projects/CellReadR/Code/Functions/output.csv', sep = ';')

In [ ]:
# Trying to load entire transcripome ... obviously failed ... 
allRat_fileName = '/home/user1/Dropbox/Research/Neurobiology_PhD/Rotations/Huang/Projects/CellReadR/Data/Sequences/Reference/Ensembl/All/Rattus_norvegicus.Rnor_6.0.cdna.all.fa'
allRat = list(SeqIO.parse(allRat_fileName, "fasta"))

In [ ]:
python -m intarnapvalue --query GCUGAAAAACAUAACCCAUAAAAUGCUAGCUGUACCAGGAACCA --target GGUUUCUUCGCCUCUGCGUUCACCAAAGUGUUCACCC --scores 10000 --shuffle-mode b --threads 0

In [ ]:
intarnapvalue --query GCUGAAAAACAUAACCCAUAAAAUGCUAGCUGUACCAGGAACCA --target GGUUUCUUCGCCUCUGCGUUCACCAAAGUGUUCACCC --scores 10000 --shuffle-mode b --threads 0

# Misc

In [ ]:
# For returning index of findings 
searchCodon = 'TAG'
[m.start() for m in re.finditer(searchCodon, str(seq))]

In [ ]:
testSeq = sesRNAs[0]
testSeq

In [ ]:
lastATG = [m.start() for m in re.finditer('ATG', str(testSeq))][-1]
lastTGG = [m.start() for m in re.finditer('TGG', str(testSeq))][-1]

In [ ]:
lastTGG

In [ ]:
lastATG

In [ ]:
lastATG < lastTGG

In [ ]:
searchCodon = 'ATG'
[m.start() for m in re.finditer(searchCodon, str(testSeq))][-1]

In [ ]:
seq.count('TAG') < 4

In [ ]:
seq[0:100].count('TAG')

In [ ]:
stopCodons = ['TAG', 'TAA', 'TGA']
stopCodons 

In [ ]:
indiciesTGG

In [ ]:
indiciesStop

In [ ]:
length = 200 
center = length/2

In [ ]:
arrayStop = np.array(indiciesStop)
arrayIndicies = np.array(indiciesTGG) 
centralTGGs = arrayIndicies[abs(arrayIndicies - center) < 10]

In [ ]:
centralTGGs

In [ ]:
np.in1d(centralTGGs,arrayStop)

In [ ]:
# Check if array contains values that are within range of values in another array 
any((min(abs(arrayStop - i)) > 10) for i in centralTGGs)

In [ ]:
centralTGGs

In [ ]:
indiciesStop

In [ ]:
testStop = [90, 16, 174]

In [ ]:
(min(abs(arrayStop - centralTGGs[0])) > 10)

In [ ]:
min(abs(arrayStop - centralTGGs[0])) > 10

In [ ]:
centeralTGGs = offset.min()
centeralTGGs

In [ ]:
centralTGGs = np.all(offset == offset.min())
centralTGGs

In [ ]:
centeralTGGs = np.where(offset == offset.min())
centeralTGG

In [ ]:
offset = abs(arrayIndicies - center) 
centerTGG = indiciesTGG[np.argmin(offset)]

In [ ]:
any(abs(x - centerTGG) < 10 for x in indiciesStop)

In [ ]:
indiciesStop = []
for codons in stopCodons:
    indiciesStop.extend([m.start() for m in re.finditer(codons, str(testSeq))])

In [ ]:
len(indiciesStop)

In [ ]:
[m.start() for m in re.finditer('TGA', str(testSeq))]

In [ ]:
[m.start() for m in re.finditer('TAA', str(testSeq))]

In [ ]:
[m.start() for m in re.finditer('TAG', str(testSeq))]

In [ ]:
testSeq.count(stopCodons)

In [ ]:
indiciesTGG

In [ ]:
len(indiciesTGG)

In [ ]:
testSeq = sesRNAs[0]

In [ ]:
lastTGG = [m.start() for m in re.finditer('TGG', str(testSeq))][-1]

In [ ]:
testSeq

In [ ]:
indiciesTGG = [m.start() for m in re.finditer('TGG', str(testSeq))]

In [ ]:
start = 0 
stop = 200

In [ ]:
middle = (start + stop) / 2

In [ ]:
abs(middle - indiciesTGG[0])

In [ ]:
type(indiciesTGG)

In [ ]:
indiciesTGG

In [ ]:
any(indiciesTGG) > 2

In [ ]:
length = 200 

In [ ]:
any(abs(x - (length/2)) < 20 for x in indiciesTGG)

In [ ]:
testList = [50, 60, 170, 200]

In [ ]:
any(abs(x - (length/2)) < 10 for x in testList)

In [ ]:
testSeq

In [ ]:
os.path.isdir('Output/BioPython')

In [ ]:
outputFileName = os.getcwd() + ''

In [ ]:
testSeq